In [75]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import seaborn
from sklearn.neural_network import MLPClassifier


In [76]:
df = pd.read_csv("./train.csv")
df = df.sample(frac = 1)
print(df.columns)
print(df['defects'][0])

Index(['id', 'loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e',
       'b', 't', 'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment',
       'uniq_Op', 'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount',
       'defects'],
      dtype='object')
False


In [77]:
def features_engineering(df):
    df['mean_bnv'] = (df['n'] + df['v'] + df['b']) /3;
    df['mean_uniqOpOpend'] = (df['uniq_Op'] + df['uniq_Opnd']) /2;
    df['mean_totOpOpend'] = (df['total_Op'] + df['total_Opnd']) /2;
    df['mean_brcntvg'] = (df['branchCount'] + df['v(g)']) / 2;
    return df

# newDF = features_engineering(df)

# Random Forest

In [78]:
X = df.drop(['defects','id'],axis=1)
y = df['defects']

X = StandardScaler().fit_transform(X)

x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
print("Train :",len(y_train))
print("Test :",len(y_test))

Train : 81410
Test : 20353


In [79]:
model = RandomForestClassifier(n_estimators = 150,
                                  criterion = 'gini',
                                  max_depth = 7,
                                  min_samples_leaf = 12,
                                  max_features  = 'sqrt',
                                  bootstrap = True,
                                  oob_score = True,
                                  n_jobs = -1, 
                                  random_state = 42,
                                  verbose = 0)
model.fit(x_train,y_train)

print(model.score(x_test,y_test))

0.813737532550484


### RandomForest :
score => 0.815
Time => 39.2s

### SVC :
score => 0.811
Time => 11 min

### DecisionTree :
score => 0.72
Time => 2.8s

### KNN :
score => 0.788
Time => 2.2s

### MLP:
score => 0.813
Time => 2.2s


# Submission

In [80]:
test_X = pd.read_csv("./test.csv").drop('id',axis=1)

test_X = StandardScaler().fit_transform(test_X)


results = model.predict_proba(test_X)[:, 1]

submission = pd.DataFrame()

submission['id'] =  np.arange(101763,101763 + len(results))   # np.arange(101763,101763 + len(results[:67842]))
submission['defects'] = results

print(submission)
submission.to_csv("./submission.csv",index=False)

           id   defects
0      101763  0.249705
1      101764  0.214442
2      101765  0.675598
3      101766  0.445325
4      101767  0.136572
...       ...       ...
67837  169600  0.300113
67838  169601  0.106787
67839  169602  0.149949
67840  169603  0.104740
67841  169604  0.567638

[67842 rows x 2 columns]
